In [7]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import math
pd.set_option('display.max_columns', None)


In [17]:
data = pd.read_csv('feature_df.csv', index_col=0)
data = data.drop(columns=['Season', 'Week', 'Date', 'Time',
       'H/A', 'W/L', 'PF', 'PA'])

In [18]:
data

,COV,COV_avg12,COV_avg16,COV_avg2,COV_avg3,COV_avg4,COV_avg6,COV_avg8,COV_lag1,COV_lag10,COV_lag11,COV_lag12,COV_lag13,COV_lag14,COV_lag15,COV_lag16,COV_lag2,COV_lag3,COV_lag4,COV_lag5,COV_lag6,COV_lag7,COV_lag8,COV_lag9,COV_opp,DEF,DEF_avg12,DEF_avg16,DEF_avg2,DEF_avg3,DEF_avg4,DEF_avg6,DEF_avg8,DEF_lag1,DEF_lag10,DEF_lag11,DEF_lag12,DEF_lag13,DEF_lag14,DEF_lag15,DEF_lag16,DEF_lag2,DEF_lag3,DEF_lag4,DEF_lag5,DEF_lag6,DEF_lag7,DEF_lag8,DEF_lag9,DEF_opp,OFF,OFF_avg12,OFF_avg16,OFF_avg2,OFF_avg3,OFF_avg4,OFF_avg6,OFF_avg8,OFF_lag1,OFF_lag10,OFF_lag11,OFF_lag12,OFF_lag13,OFF_lag14,OFF_lag15,OFF_lag16,OFF_lag2,OFF_lag3,OFF_lag4,OFF_lag5,OFF_lag6,OFF_lag7,OFF_lag8,OFF_lag9,OFF_opp,OVR,OVR_avg12,OVR_avg16,OVR_avg2,OVR_avg3,OVR_avg4,OVR_avg6,OVR_avg8,OVR_lag1,OVR_lag10,OVR_lag11,OVR_lag12,OVR_lag13,OVR_lag14,OVR_lag15,OVR_lag16,OVR_lag2,OVR_lag3,OVR_lag4,OVR_lag5,OVR_lag6,OVR_lag7,OVR_lag8,OVR_lag9,OVR_opp,Opposing Team,PASS,PASS_avg12,PASS_avg16,PASS_avg2,PASS_avg3,PASS_avg4,PASS_avg6,PASS_avg8,PASS_lag1,PASS_lag10,PASS_lag11,PASS_lag12,PASS_lag13,PASS_lag14,PASS_lag15,PASS_lag16,PASS_lag2,PASS_lag3,PASS_lag4,PASS_lag5,PASS_lag6,PASS_lag7,PASS_lag8,PASS_lag9,PASS_opp,PBLK,PBLK_avg12,PBLK_avg16,PBLK_avg2,PBLK_avg3,PBLK_avg4,PBLK_avg6,PBLK_avg8,PBLK_lag1,PBLK_lag10,PBLK_lag11,PBLK_lag12,PBLK_lag13,PBLK_lag14,PBLK_lag15,PBLK_lag16,PBLK_lag2,PBLK_lag3,PBLK_lag4,PBLK_lag5,PBLK_lag6,PBLK_lag7,PBLK_lag8,PBLK_lag9,PBLK_opp,PRSH,PRSH_avg12,PRSH_avg16,PRSH_avg2,PRSH_avg3,PRSH_avg4,PRSH_avg6,PRSH_avg8,PRSH_lag1,PRSH_lag10,PRSH_lag11,PRSH_lag12,PRSH_lag13,PRSH_lag14,PRSH_lag15,PRSH_lag16,PRSH_lag2,PRSH_lag3,PRSH_lag4,PRSH_lag5,PRSH_lag6,PRSH_lag7,PRSH_lag8,PRSH_lag9,PRSH_opp,RBLK,RBLK_avg12,RBLK_avg16,RBLK_avg2,RBLK_avg3,RBLK_avg4,RBLK_avg6,RBLK_avg8,RBLK_lag1,RBLK_lag10,RBLK_lag11,RBLK_lag12,RBLK_lag13,RBLK_lag14,RBLK_lag15,RBLK_lag16,RBLK_lag2,RBLK_lag3,RBLK_lag4,RBLK_lag5,RBLK_lag6,RBLK_lag7,RBLK_lag8,RBLK_lag9,RBLK_opp,RDEF,RDEF_avg12,RDEF_avg16,RDEF_avg2,RDEF_avg3,RDEF_avg4,RDEF_avg6,RDEF_avg8,RDEF_lag1,RDEF_lag10,RDEF_lag11,RDEF_lag12,RDEF_lag13,RDEF_lag14,RDEF_lag15,RDEF_lag16,RDEF_lag2,RDEF_lag3,RDEF_lag4,RDEF_lag5,RDEF_lag6,RDEF_lag7,RDEF_lag8,RDEF_lag9,RDEF_opp,RECV,RECV_avg12,RECV_avg16,RECV_avg2,RECV_avg3,RECV_avg4,RECV_avg6,RECV_avg8,RECV_lag1,RECV_lag10,RECV_lag11,RECV_lag12,RECV_lag13,RECV_lag14,RECV_lag15,RECV_lag16,RECV_lag2,RECV_lag3,RECV_lag4,RECV_lag5,RECV_lag6,RECV_lag7,RECV_lag8,RECV_lag9,RECV_opp,RUN,RUN_avg12,RUN_avg16,RUN_avg2,RUN_avg3,RUN_avg4,RUN_avg6,RUN_avg8,RUN_lag1,RUN_lag10,RUN_lag11,RUN_lag12,RUN_lag13,RUN_lag14,RUN_lag15,RUN_lag16,RUN_lag2,RUN_lag3,RUN_lag4,RUN_lag5,RUN_lag6,RUN_lag7,RUN_lag8,RUN_lag9,RUN_opp,TACK,TACK_avg12,TACK_avg16,TACK_avg2,TACK_avg3,TACK_avg4,TACK_avg6,TACK_avg8,TACK_lag1,TACK_lag10,TACK_lag11,TACK_lag12,TACK_lag13,TACK_lag14,TACK_lag15,TACK_lag16,TACK_lag2,TACK_lag3,TACK_lag4,TACK_lag5,TACK_lag6,TACK_lag7,TACK_lag8,TACK_lag9,TACK_opp,Team,gamenum
256,62.1,61.166667,61.98125,51.40,52.433333,55.525,63.500000,59.2875,47.4,61.9,57.1,64.2,67.0,74.0,58.9,57.8,55.4,54.5,64.8,79.2,79.7,45.7,47.6,76.5,91.4,54.1,67.766667,67.66250,64.80,64.066667,64.300,68.566667,64.6000,65.7,72.6,63.7,78.5,59.9,73.5,68.6,67.4,63.9,62.6,65.0,81.8,72.4,57.4,48.0,81.6,85.3,51.4,72.408333,72.20625,66.40,65.366667,68.000,69.633333,70.5875,62.1,70.9,72.2,78.9,74.1,81.9,63.5,66.9,70.7,63.3,75.9,70.7,75.1,73.4,73.5,82.2,92.5,51.0,72.808333,72.56250,67.20,66.000000,68.000,71.000000,70.1125,64.5,73.9,70.5,82.4,70.2,81.7,67.0,68.4,69.9,63.6,74.0,77.5,76.5,69.6,65.3,86.0,92.2,Indianapolis Colts,45.1,75.000000,72.34375,71.30,65.666667,69.400,71.150000,72.5125,51.3,75.7,67.4,91.2,76.7,68.0,54.2,58.6,91.3,54.4,80.6,69.7,79.6,74.1,79.1,85.6,94.4,61.7,77.783333,77.55000,60.30,69.866667,72.575,73.900000,74.8625,55.9,84.6,79.2,82.9,85.3,57.2,78.7,86.2,64.7,89.0,80.7,88.0,65.1,76.3,79.2,87.8,72.7,60.8,61.633333,61.01875,56.10,55.933333,58.175,58.733333,58.8125,48.9,54.8,66.1,80.1,57.7,63.3,56.3,59.4,63.3,55.6,64.9,61.4,58.3,61.3,56.

In [ ]:
%%time
# feature_importances_
pp = pprint.PrettyPrinter(indent=4)
for attribute in attributes:
    # Lists to hold on to the generated values
    gbr_preds = []
    bagr_preds = []
    t = []
    teams = []
    games = []
    
    columns = []
    
    if attribute == 'OVR':
        # lags
        for i in range(1, 17):
            columns.append('OVR_lag' + str(i))
            columns.append('OFF_lag' + str(i))
            columns.append('PASS_lag' + str(i))
            columns.append('RECV_lag' + str(i))
            
        # averages
        for j in (2,3,4,6,8,12,16):
            columns.append('OVR_avg' + str(j))
            # opposing
            columns.append('OVR_avg' + str(j) + '_opposing')
            columns.append('DEF_avg' + str(j) + '_opposing')
            columns.append('COV_avg' + str(j) + '_opposing')
        
    elif attribute == 'OFF':
        #lags
        for i in range(1, 17):
            columns.append('OFF_lag' + str(i))
            columns.append('PASS_lag' + str(i))
            columns.append('RECV_lag' + str(i))
            columns.append('PBLK_lag' + str(i))
            columns.append('RBLK_lag' + str(i))
            columns.append('RUN_lag' + str(i))
            
        # averages
        for j in (2,3,4,6,8,12,16):
            columns.append('OFF_avg' + str(j))
            columns.append('PASS_avg' + str(j))
            columns.append('RECV_avg' + str(j))
            columns.append('PBLK_avg' + str(j))
            columns.append('RBLK_avg' + str(j))
            columns.append('RUN_avg' + str(j))
            # opposing
            columns.append('DEF_avg' + str(j) + '_opposing')
            columns.append('COV_avg' + str(j) + '_opposing')
            columns.append('RDEF_avg' + str(j) + '_opposing')
            columns.append('PRSH_avg' + str(j) + '_opposing')

    elif attribute == 'DEF':
        # lags
        for i in range(1, 17):
            columns.append('DEF_lag' + str(i))
            columns.append('OVR_lag' + str(i))
            columns.append('PRSH_lag' + str(i))
            columns.append('RDEF_lag' + str(i))
            columns.append('COV_lag' + str(i))
            columns.append('TACK_lag' + str(i))
            
        # averages
        for j in (2,3,4,6,8,12,16):
            columns.append('DEF_avg' + str(j))
            columns.append('OVR_avg' + str(j))
            columns.append('PRSH_avg' + str(j))
            columns.append('RDEF_avg' + str(j))
            columns.append('COV_avg' + str(j))
            columns.append('TACK_avg' + str(j))
            # opposing
            columns.append('OFF_avg' + str(j) + '_opposing')
            columns.append('PASS_avg' + str(j) + '_opposing')
            columns.append('RBLK_avg' + str(j) + '_opposing')
            columns.append('RECV_avg' + str(j) + '_opposing')
            columns.append('PBLK_avg' + str(j) + '_opposing')

    elif attribute == 'RBLK':
        # lags
        for i in range(1, 17):
            columns.append('RBLK_lag' + str(i))
            columns.append('PBLK_lag' + str(i))
        
        # averages
        for j in (2,3,4,6,8,12,16):
            columns.append('RBLK_avg' + str(j))
            columns.append('PBLK_avg' + str(j))
            # opposing
            columns.append('RDEF_avg' + str(j) + '_opposing')
            columns.append('COV_avg' + str(j) + '_opposing')
            columns.append('PRSH_avg' + str(j) + '_opposing')

    elif attribute == 'PBLK':
        # lags
        for i in range(1, 17):
            columns.append('PBLK_lag' + str(i))
            
        # averages
        for j in (2,3,4,6,8,12,16):
            columns.append('PBLK_avg' + str(j))
            # opposing
            columns.append('PRSH_avg' + str(j) + '_opposing')
    
    elif attribute == 'RECV':
        # lags
        for i in range(1, 17):
            columns.append('RECV_lag' + str(i))
            columns.append('PASS_lag' + str(i))
            
        # averages
        for j in (2,3,4,6,8,12,16):
            columns.append('RECV_avg' + str(j))
            columns.append('PASS_avg' + str(j))
            # opposing
            columns.append('TACK_avg' + str(j) + '_opposing')
            columns.append('PRSH_avg' + str(j) + '_opposing')
            columns.append('COV_avg' + str(j) + '_opposing')

    elif attribute == 'PASS':
        # lags
        for i in range(1, 17):
            columns.append('PASS_lag' + str(i))
            columns.append('RECV_lag' + str(i))
            columns.append('PBLK_lag' + str(i))
            
        # averages
        for j in (2,3,4,6,8,12,16):
            columns.append('PASS_avg' + str(j))
            columns.append('RECV_avg' + str(j))
            columns.append('PBLK_avg' + str(j))
            # opposing
            columns.append('PRSH_avg' + str(j) + '_opposing')
            columns.append('COV_avg' + str(j) + '_opposing')

    elif attribute == 'RUN':
        # lags
        for i in range(1, 17):
            columns.append('RBLK_lag' + str(i))
            columns.append('RUN_lag' + str(i))
            
        # averages
        for j in (2,3,4,6,8,12,16):
            columns.append('RBLK_avg' + str(j))
            columns.append('RUN_avg' + str(j))
            # opposing
            columns.append('TACK_avg' + str(j) + '_opposing')
            columns.append('RDEF_avg' + str(j) + '_opposing')

    elif attribute == 'TACK':
        # lags
        for i in range(1, 17):
            columns.append('TACK_lag' + str(i))
            columns.append('RDEF_lag' + str(i))
            
        # averages
        for j in (2,3,4,6,8,12,16):
            columns.append('TACK_avg' + str(j))
            columns.append('RDEF_avg' + str(j))
            # opposing
            columns.append('RECV_avg' + str(j) + '_opposing')
            columns.append('RUN_avg' + str(j) + '_opposing')

    elif attribute == 'RDEF':
        # lags
        for i in range(1, 17):
            columns.append('TACK_lag' + str(i))
            columns.append('RDEF_lag' + str(i))
            
        # averages
        for j in (2,3,4,6,8,12,16):
            columns.append('TACK_avg' + str(j))
            columns.append('RDEF_avg' + str(j))
            # opposing
            columns.append('RBLK_avg' + str(j) + '_opposing')
            columns.append('RUN_avg' + str(j) + '_opposing')

    elif attribute == 'PRSH':
        # lags
        for i in range(1, 17):
            columns.append('PRSH_lag' + str(i))
            
        # averages
        for j in (2,3,4,6,8,12,16):
            columns.append('PRSH_avg' + str(j))
            # opposing
            columns.append('PBLK_avg' + str(j) + '_opposing')
            columns.append('PASS_avg' + str(j) + '_opposing')
            columns.append('RECV_avg' + str(j) + '_opposing')

    elif attribute == 'COV':
        # lags
        for i in range(1, 17):
            columns.append('PRSH_lag' + str(i))
            columns.append('COV_lag' + str(i))
            columns.append('TACK_lag' + str(i))
            
        # averages
        for j in (2,3,4,6,8,12,16):
            columns.append('PRSH_avg' + str(j))
            columns.append('COV_avg' + str(j))
            columns.append('TACK_avg' + str(j))
            # opposing
            columns.append('PBLK_avg' + str(j) + '_opposing')
            columns.append('RECV_avg' + str(j) + '_opposing')
            columns.append('PASS_avg' + str(j) + '_opposing')

    for team in grouped:
        data = team[1]
        
        
        # Train test split
        train = data[data['Season'] < 2017]
        test = data[data['Season'] >= 2017]
        
        y_train = train[attribute]
        X_train = train[columns]
        
        y_test = test[attribute]
        X_test = test[columns]
        
        team = test['Team'].values
        gameNum = test['GameNum'].values
        
        
        # Gradient Boosting Regressor
        gbr = GradientBoostingRegressor()
        
        # Bagging Regressor
        bag = BaggingRegressor()
        
        # Train models
        gbr.fit(X_train, y_train)
        bag.fit(X_train, y_train)
        
        
        
        # Now Cross Validate
        grid_search = GridSearchCV(gbr, gbr_parameters, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=3)
        grid_search.fit(X_train, y_train)
        
        print(attribute, "GBR", grid_search.best_params_)

        gbr_regressor = GradientBoostingRegressor(
            loss=grid_search.best_params_['loss'],
            max_depth=grid_search.best_params_['max_depth'],
            n_estimators=grid_search.best_params_['n_estimators'],
            learning_rate=grid_search.best_params_['learning_rate']
        )
        
        
        #Save best model parameters
        best_boost = best_boost.append({'Attribute' : attribute,
                                        'Team' : team[0],
                                        'loss' : grid_search.best_params_['loss'],
                                        'learning_rate' : grid_search.best_params_['learning_rate'],
                                        'max_depth' : grid_search.best_params_['max_depth'],
                                        'n_estimators' : grid_search.best_params_['n_estimators']},
                                      ignore_index = True)
        
        
        
        # CV BAG
        grid_search = GridSearchCV(bag, bag_parameters, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=3)
        grid_search.fit(X_train, y_train)
        
        print(attribute, "BAG", grid_search.best_params_)

        bag_regressor = BaggingRegressor(
            
            max_samples=grid_search.best_params_['max_samples'],
            n_estimators=grid_search.best_params_['n_estimators']
        )
        
        # Save the best model parameters
        best_bag = best_bag.append({'Attribute' : attribute,
                                    'Team' : team[0],
                                    'max_samples' : grid_search.best_params_['max_samples'],
                                    'n_estimators' : grid_search.best_params_['n_estimators']},
                                  ignore_index = True)
        
        
        # Train cross validated models
        gbr_regressor.fit(X_train, y_train)
        bag_regressor.fit(X_train, y_train)
        
        # feature_importance
        features = list(X_train.columns)
        gbr_regressor_importances = list(gbr_regressor.feature_importances_)
        gbr_regressor_feature_importance = dict.fromkeys(features, 0)
        i = 0
        for key in gbr_regressor_feature_importance.keys():
            gbr_regressor_feature_importance[key] = gbr_regressor_importances[i]
            i += 1
        variable_importances.append([attribute, team[0], gbr_regressor_feature_importance])
        
        # Make predictions
        gbr_hat = gbr_regressor.predict(X_test)
        bagr_hat = bag_regressor.predict(X_test)
        
        
        # Save each team's data
        for i in range(len(y_test)):
            t.append(y_test.iloc[i])
            gbr_preds.append(gbr_hat[i])
            bagr_preds.append(bagr_hat[i])
            teams.append(team[i])
            games.append(gameNum[i])
    
    # Save data for each attribute    
    gbr_predictions[attribute] = gbr_preds
    bagr_predictions[attribute] = bagr_preds
    target[attribute] = t
    
    gbr_predictions['Team'] = teams
    gbr_predictions['GameNum'] = games
    bagr_predictions['Team'] = teams
    bagr_predictions['GameNum'] = games
    target['Team'] = teams
    target['GameNum'] = games
    
    # Takes a while to run, so this shows progress
    print('\n\n\n\n\n\nFinished creating preditcions for:', attribute, '\n\n\n\n\n\n\n')